# 레드마인 활용 생성형 AI(ChatGPT)와 RAG 연동 POC

> 

- 레드마인 문서 자연어 처리 ChatBot
- LangChain
- ChatGPT (gpt-3.5-turbo)
- ChromaDB

> https://partner.medialog.co.kr/projects/rag-gen-ai-chatgpt-poc


In [5]:
# 주피터 랩 시스템 확인 
!pwd
!ls -al
!date
!hostnamectl
!nvidia-smi  #현재 내부 개발장비에 GPU 부착되어 있지 않음

/home/msg-platfm/redmine-chatbot
total 172
drwxrwxr-x  6 msg-platfm msg-platfm  4096  4월  8 18:36 .
drwxr-x--- 11 msg-platfm msg-platfm  4096  4월  8 18:27 ..
drwxrwxr-x  3 msg-platfm msg-platfm  4096  4월  8 11:37 articles
drwxrwxr-x  3 msg-platfm msg-platfm  4096  4월  8 11:37 articles_redmine
drwxrwxr-x  3 msg-platfm msg-platfm  4096  4월  8 17:59 db
drwxrwxr-x  2 msg-platfm msg-platfm  4096  4월  8 10:57 .ipynb_checkpoints
-rw-r--r--  1 msg-platfm msg-platfm 68750  4월  8 18:36 redmine-chatbot.ipynb
-rw-rw-r--  1 msg-platfm msg-platfm 79974  4월  8 10:20 techcrunch-articles.zip
2024. 04. 08. (월) 20:45:57 KST
 Static hostname: msgplatfm-virtual-machine
       Icon name: computer-vm
         Chassis: vm
      Machine ID: 1be3213771fc4a9e9432ce9675c63136
         Boot ID: 2d6d7d821f6148b884d124f1ecd01a0b
  Virtualization: vmware
Operating System: ]8;;https://www.ubuntu.com/Ubuntu 22.04.4 LTS]8;;              
          Kernel: Linux 6.5.0-26-generic
    Architecture: x86-64
 Hardware Ven

In [13]:
#!pip install -q langchain openai tiktoken chromadb

## 외부 문서 불러와서  로컬에 저장 테스트

> TechCrunch 기사 21개

In [14]:
#!wget -q https://github.com/kairess/toy-datasets/raw/master/techcrunch-articles.zip
#!unzip -q techcrunch-articles.zip -d articles

## 레드마인 문서 가져 와서 파일별로 저장하기

In [13]:
!pip install requests beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable


In [25]:
import requests
from bs4 import BeautifulSoup  # 이 부분을 수정했습니다.

import os
REDMINE_API_KEY = os.getenv("REDMINE_API_KEY")

# 레드마인 서버 설정
REDMINE_URL = 'https://partner.medialog.co.kr'
REDMINE_API_KEY = os.getenv("REDMINE_API_KEY")
PROJECT_ID = 'rag-gen-ai-chatgpt-poc'

# 위키 페이지 목록을 가져오는 URL
WIKI_INDEX_URL = f"{REDMINE_URL}/projects/{PROJECT_ID}/wiki/index.json"

headers = {
    "X-Redmine-API-Key": REDMINE_API_KEY
}

def get_wiki_pages():
    """위키 페이지 목록을 가져옵니다."""
    response = requests.get(WIKI_INDEX_URL, headers=headers, verify=False)  # SSL 인증서 검증을 무시
    if response.status_code == 200:
        data = response.json()
        # API 응답 구조에 따라 적절한 키를 사용하여 실제 페이지 목록을 반환
        return data.get('wiki_pages', [])  # 'wiki_pages'는 예시 키입니다. 실제 키는 API 문서를 참조하세요.
    else:
        print(f"Error fetching wiki pages: {response.status_code}")
        return []
def save_wiki_page(title, content):
    """위키 페이지 내용을 db 폴더에 파일로 저장합니다."""
    # 'db' 폴더 경로 설정
    folder_path = 'articles_redmine'
    
    # 'db' 폴더가 없으면 생성
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 파일 경로를 폴더 경로와 파일 이름으로 구성
    filename = os.path.join(folder_path, f"{title}.txt")
    
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(content)
    print(f"Saved: {filename}")

def main():
    wiki_pages = get_wiki_pages()
    
    for page in wiki_pages:
        page_url = f"{REDMINE_URL}/projects/{PROJECT_ID}/wiki/{page['title']}.json"
        # 여기에 verify=False 옵션을 추가했습니다.
        response = requests.get(page_url, headers=headers, verify=False)


        if response.status_code == 200:
            page_data = response.json()
            # 'text'에 접근하기 전에 'wiki_page' 키를 사용
            soup = BeautifulSoup(page_data['wiki_page']['text'], 'html.parser')
            text_without_tags = soup.get_text(separator='\n', strip=True)
            save_wiki_page(page_data['wiki_page']['title'], text_without_tags)
        else:
            print(f"Error fetching page {page['title']}: {response.status_code}")

    
      
if __name__ == "__main__":
    main()


f3ebd802a2b5b097021f31c3628af7b9e2191f1b
Saved: articles_redmine/Redmine.txt
Saved: articles_redmine/Wiki.txt


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'partner.medialog.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'partner.medialog.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'partner.medialog.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRe

Saved: articles_redmine/기업솔루션사업담당소개.txt
Saved: articles_redmine/도서.txt
Saved: articles_redmine/메시징플랫폼개발팀원_페르소나.txt
Saved: articles_redmine/문서.txt


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'partner.medialog.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'partner.medialog.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'partner.medialog.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRe

Saved: articles_redmine/미디어로그회사소개.txt


## Setting up LangChain
OpenAI APIKey  확인
https://platform.openai.com/account/api-keys   

In [26]:
import os
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key is not None:
    print("OpenAI API Key found:")
else:
    print("OpenAI API Key not found.")

OpenAI API Key found:


In [27]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

## Load multiple and process documents

In [47]:
#loader = TextLoader('single_text_file.txt')   #싱글 파일 인 경우
loader = DirectoryLoader('./articles_redmine', glob="*.txt", loader_cls=TextLoader)  

documents = loader.load()

len(documents)
print(documents)

[Document(page_content='레드마인\n레드마인은 미디어로그에서 활용하고 있는\xa0 PMS 도구 입니다.\n유플러스와의\xa0 SM / SI\xa0 업무\xa0 시 Task 관리에 유용하게 사용되고 있고\xa0 슬랙은 업무간 소통도구로 활용되고 있습니다.', metadata={'source': 'articles_redmine/Redmine.txt'}), Document(page_content='{{include(문서)}}', metadata={'source': 'articles_redmine/Wiki.txt'}), Document(page_content='메시징플랫폼개발팀원 페르소나\n최성순님은\xa0 메시징플랫폼개발팀을 맡고 있고\xa0 클라우드 네이티브 방식으로 기업메시징 중계서비스를 운영하고 싶어합니다.\n최근에는 생성형 AI 에 관심을 가지고\xa0 "레드마인 활용 생성형 AI(ChatGPT)와 RAG 연동 POC" 를\xa0 진행하고 있고\xa0 팀원들의\xa0 생성형 AI 활용도를 높이기 위하여\xa0 GPT Team 구독을 시작하였습니다.\n김태균님은 메시징플랫폼개발팀에서 메시지허브 운영을 하고 있습니다.\n메시징 서비스 경험은 이번이 처음이지만 메시지허브를 빨리 장악하고 싶어합니다.\n메시지허브에 AI를 적용할 수 있는 방안을 모색하고 나아가서는 통합 사이트 구축 시 도움이 되길 바라고 있습니다.\n황봉규님은 기업메시징 클라이언트개발운영 및 기술지원 업무를 하고 있습니다.\n주요업무는 고객서버에 설치되는 클라이언트의 개발 및 오류 수정, 분석/설계,개발 투입공수 산정등입니다.\n클라이언트는 sms,mms,rcs,알림톡등 핸드폰에서 수신가능한 메시지를 발송하는 자바로 만들어진 프로그램입니다.\n기술지원은 다양한 고객 서버 환경에 클라이언트를 설치하고 장애대응을 해줍니다.\n자바,스프링부트,다양한 os, 다양한 db 에 대한 기초 지식이 필요합니다.\n조용필님은 기업메시징 엔진운영 업무를 하고 있습니다.\nHadoop EcoSyste

## Split texts

In [48]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

len(texts)
#print(texts)

[Document(page_content='레드마인\n레드마인은 미디어로그에서 활용하고 있는\xa0 PMS 도구 입니다.\n유플러스와의\xa0 SM / SI\xa0 업무\xa0 시 Task 관리에 유용하게 사용되고 있고\xa0 슬랙은 업무간 소통도구로 활용되고 있습니다.', metadata={'source': 'articles_redmine/Redmine.txt'}), Document(page_content='{{include(문서)}}', metadata={'source': 'articles_redmine/Wiki.txt'}), Document(page_content='메시징플랫폼개발팀원 페르소나\n최성순님은\xa0 메시징플랫폼개발팀을 맡고 있고\xa0 클라우드 네이티브 방식으로 기업메시징 중계서비스를 운영하고 싶어합니다.\n최근에는 생성형 AI 에 관심을 가지고\xa0 "레드마인 활용 생성형 AI(ChatGPT)와 RAG 연동 POC" 를\xa0 진행하고 있고\xa0 팀원들의\xa0 생성형 AI 활용도를 높이기 위하여\xa0 GPT Team 구독을 시작하였습니다.\n김태균님은 메시징플랫폼개발팀에서 메시지허브 운영을 하고 있습니다.\n메시징 서비스 경험은 이번이 처음이지만 메시지허브를 빨리 장악하고 싶어합니다.\n메시지허브에 AI를 적용할 수 있는 방안을 모색하고 나아가서는 통합 사이트 구축 시 도움이 되길 바라고 있습니다.\n황봉규님은 기업메시징 클라이언트개발운영 및 기술지원 업무를 하고 있습니다.\n주요업무는 고객서버에 설치되는 클라이언트의 개발 및 오류 수정, 분석/설계,개발 투입공수 산정등입니다.\n클라이언트는 sms,mms,rcs,알림톡등 핸드폰에서 수신가능한 메시지를 발송하는 자바로 만들어진 프로그램입니다.\n기술지원은 다양한 고객 서버 환경에 클라이언트를 설치하고 장애대응을 해줍니다.\n자바,스프링부트,다양한 os, 다양한 db 에 대한 기초 지식이 필요합니다.\n조용필님은 기업메시징 엔진운영 업무를 하고 있습니다.\nHadoop EcoSyste

In [39]:
texts[2:4]

[Document(page_content='메시징플랫폼개발팀원 페르소나\n최성순님은\xa0 메시징플랫폼개발팀을 맡고 있고\xa0 클라우드 네이티브 방식으로 기업메시징 중계서비스를 운영하고 싶어합니다.\n최근에는 생성형 AI 에 관심을 가지고\xa0 "레드마인 활용 생성형 AI(ChatGPT)와 RAG 연동 POC" 를\xa0 진행하고 있고\xa0 팀원들의\xa0 생성형 AI 활용도를 높이기 위하여\xa0 GPT Team 구독을 시작하였습니다.\n김태균님은 메시징플랫폼개발팀에서 메시지허브 운영을 하고 있습니다.\n메시징 서비스 경험은 이번이 처음이지만 메시지허브를 빨리 장악하고 싶어합니다.\n메시지허브에 AI를 적용할 수 있는 방안을 모색하고 나아가서는 통합 사이트 구축 시 도움이 되길 바라고 있습니다.\n황봉규님은 기업메시징 클라이언트개발운영 및 기술지원 업무를 하고 있습니다.\n주요업무는 고객서버에 설치되는 클라이언트의 개발 및 오류 수정, 분석/설계,개발 투입공수 산정등입니다.\n클라이언트는 sms,mms,rcs,알림톡등 핸드폰에서 수신가능한 메시지를 발송하는 자바로 만들어진 프로그램입니다.\n기술지원은 다양한 고객 서버 환경에 클라이언트를 설치하고 장애대응을 해줍니다.\n자바,스프링부트,다양한 os, 다양한 db 에 대한 기초 지식이 필요합니다.\n조용필님은 기업메시징 엔진운영 업무를 하고 있습니다.\nHadoop EcoSystem 솔루션 자료 조사 중으로 Big Data 시스템 구축을 하고자 함.\n이동언님은 메시징플랫폼 개발 서비스를 맡고 있고 대용량 메시징 서비스 개발 및 운영 경험을 갖고 있습니다.\n최근에는 클라우드 서비스 환경 기반 메시징 시스템 구축에 관심을 가지고 있고 생산형 AI를 통한 개발 효율화를 이루고 싶어 합니다.\n팀원들과 생성형 AI 활용 방안을 공유하고 실제 서비스에 적용할 수 있도록 개인 역량 강화 진행 중입니다.\n조재원님은 기업메시징 RCS서비스 엔진운영 업무를 하고 있습니다.\nAWS를 활용한 Cloud 기반으로 서비스

## Create Chroma DB

1. Text -> Embbedings
2. `db` 폴더에 데이터 저장
3. DB 초기화
4. `db` 폴더로부터 DB 로드

In [49]:
persist_directory = 'db'

embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=persist_directory)

In [50]:
vectordb.persist()
vectordb = None

In [51]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding)

## Make a retriever

In [52]:
retriever = vectordb.as_retriever()

In [61]:
#docs = retriever.get_relevant_documents("최성순은 어떤 사람?")
docs = retriever.get_relevant_documents("미디어로그 대표는?")

for doc in docs:
    print(doc.metadata["source"])

articles_redmine/미디어로그회사소개.txt
articles_redmine/미디어로그회사소개.txt
articles_redmine/members.txt
articles_redmine/members.txt


### 결과를 k개 반환

In [42]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [44]:
docs = retriever.get_relevant_documents("신귀매는?")

for doc in docs:
    print(doc.metadata["source"])

articles_redmine/Wiki.txt
arti_partner/members.txt
articles_redmine/members.txt


## Make a chain

In [62]:
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

In [63]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

## Query

In [64]:
#query = "How much money did Pando raise?"
query = "미디어로그 대표는 ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 이상헌


Sources:
articles_redmine/미디어로그회사소개.txt
articles_redmine/미디어로그회사소개.txt
articles_redmine/members.txt
articles_redmine/members.txt


In [65]:
llm_response

{'query': '미디어로그 대표는 ?',
 'result': ' 이상헌',
 'source_documents': [Document(page_content='미디어로그회사소개\nMVNO 영역은\xa0\xa0기존\xa0통신사의\xa0통신비\xa0거품을\xa0없애, 5G/LTE\xa0서비스를\xa0합리적인\xa0요금으로\xa0제공합니다.\xa0 LG\xa0U\n+\n와\xa0동일한\xa0데이터\xa0품질을\xa0제공하며\xa0실속있는\xa0상품으로\xa0통신비를\xa0절약해드립니다. 다양한\xa0폰을\xa0합리적인\xa0가격에\xa0구매할\xa0수\xa0있고,\xa0알뜰한\xa0요금제로\xa0스마트한\xa0통신\xa0문화를\xa0선도합니다.\n중고폰사업 영역은\xa0 정교한\xa0성능/외관\xa0검사로\xa0가격\xa0변동을\xa0최소화하는\xa0투명성과\xa0개인정보\xa0이중\xa0삭제를\xa0통해\xa0신뢰할\xa0수\xa0있는\xa0온라인\xa0중고폰\xa0매입\xa0플랫폼(셀로)을 제공합니다. 또한,\xa0중고폰\xa0매입을\xa0넘어\xa0‘가치의\xa0선순환’을\xa0통해 더\xa0좋은\xa0공유를\xa0만들어가는\xa0사회적\xa0가치\xa0실현을\xa0위한\xa0ESG\xa0경영을\xa0실천합니다.\nMEDIA 영역은 LG\xa0U\n+\nIPTV,\xa0모바일\xa0콘텐츠의\xa0디지털\xa0변환,\xa0메타정보,\xa0노출\xa0편성,\xa0콘텐츠\xa0프로모션을\xa0수행하고\xa0있고,\xa0U\n+\ntv/모바일tv\xa0등의\xa0다양한\xa0미디어\xa0플랫폼을\xa0운영하고\xa0있습니다.\nICT 영역은 UI부터\xa0서버\xa0구축에\xa0이르는\xa0ICT\xa0기술\xa0토탈\xa0라인업을\xa0구축하고\xa0있으며\xa0U\n+\n통신\xa0플랫폼에서\xa0최고의\xa0서비스를\xa0제공합니다.\n회사명\n㈜미디어로그\n설립년월\n2000년 3월\n대표이사\n이상헌\n매출/영업이익\n2,953억 / 51억(2023년 기준)

In [31]:
query = "GPT를 잘 사용하는 사람은 ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 최성순님, 이동언님


Sources:
arti_partner/members.txt
arti_partner/members.txt
arti_partner/members.txt


In [34]:
llm_response

{'query': 'GPT를 잘 사용하는 사람은 ?',
 'result': ' 최성순님, 이동언님',
 'source_documents': [Document(page_content='최성순님은  메시징플랫폼개발팀을 맡고 있고  클라우드 네이티브 방식으로 기업메시징 중계서비스를 운영하고 싶어합니다.\n최근에는 생성형 AI 에 관심을 가지고  "레드마인 활용 생성형 AI(ChatGPT)와 RAG 연동 POC" 를  진행하고 있고  팀원들의  생성형 AI 활용도를 높이기 위하여  GPT Team 구독을 시작하였습니다.\n\n김태균님은 메시징플랫폼개발팀에서 메시지허브 운영을 하고 있습니다.\n메시징 서비스 경험은 이번이 처음이지만 메시지허브를 빨리 장악하고 싶어합니다.\n메시지허브에 AI를 적용할 수 있는 방안을 모색하고 나아가서는 통합 사이트 구축 시 도움이 되길 바라고 있습니다.\n\n\n황봉규님은 기업메시징 클라이언트개발운영 및 기술지원 업무를 하고 있습니다.\n주요업무는 고객서버에 설치되는 클라이언트의 개발 및 오류 수정, 분석/설계,개발 투입공수 산정등입니다.\n클라이언트는 sms,mms,rcs,알림톡등 핸드폰에서 수신가능한 메시지를 발송하는 자바로 만들어진 프로그램입니다.\n기술지원은 다양한 고객 서버 환경에 클라이언트를 설치하고 장애대응을 해줍니다.\n자바,스프링부트,다양한 os, 다양한 db 에 대한 기초 지식이 필요합니다.\n\n\n조용필님은 기업메시징 엔진운영 업무를 하고 있습니다.\nHadoop EcoSystem 솔루션 자료 조사 중으로 Big Data 시스템 구축을 하고자 함.\n\n\n이동언님은 메시징플랫폼 개발 서비스를 맡고 있고 대용량 메시징 서비스 개발 및 운영 경험을 갖고 있습니다.\n최근에는 클라우드 서비스 환경 기반 메시징 시스템 구축에 관심을 가지고 있고 생산형 AI를 통한 개발 효율화를 이루고 싶어 합니다. \n팀원들과 생성형 AI 활용 방안을 공유하고 실제 서비스에 적용할 수 있도록 개인 역량 강화 진행 중입니다.', metad

In [37]:
query = "Who is 최성순?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 최성순 is the person in charge of the messaging platform development team and is interested in operating enterprise messaging relay services through a cloud-native approach. They are also working on utilizing AI and have subscribed to the GPT Team to increase the team's utilization of generative AI.


Sources:
arti_partner/members.txt
arti_partner/members.txt
arti_partner/members.txt


In [38]:
llm_response

{'query': 'Who is 최성순?',
 'result': " 최성순 is the person in charge of the messaging platform development team and is interested in operating enterprise messaging relay services through a cloud-native approach. They are also working on utilizing AI and have subscribed to the GPT Team to increase the team's utilization of generative AI.",
 'source_documents': [Document(page_content='최성순님은  메시징플랫폼개발팀을 맡고 있고  클라우드 네이티브 방식으로 기업메시징 중계서비스를 운영하고 싶어합니다.\n최근에는 생성형 AI 에 관심을 가지고  "레드마인 활용 생성형 AI(ChatGPT)와 RAG 연동 POC" 를  진행하고 있고  팀원들의  생성형 AI 활용도를 높이기 위하여  GPT Team 구독을 시작하였습니다.\n\n김태균님은 메시징플랫폼개발팀에서 메시지허브 운영을 하고 있습니다.\n메시징 서비스 경험은 이번이 처음이지만 메시지허브를 빨리 장악하고 싶어합니다.\n메시지허브에 AI를 적용할 수 있는 방안을 모색하고 나아가서는 통합 사이트 구축 시 도움이 되길 바라고 있습니다.\n\n\n황봉규님은 기업메시징 클라이언트개발운영 및 기술지원 업무를 하고 있습니다.\n주요업무는 고객서버에 설치되는 클라이언트의 개발 및 오류 수정, 분석/설계,개발 투입공수 산정등입니다.\n클라이언트는 sms,mms,rcs,알림톡등 핸드폰에서 수신가능한 메시지를 발송하는 자바로 만들어진 프로그램입니다.\n기술지원은 다양한 고객 서버 환경에 클라이언트를 설치하고 장애대응을 해줍니다.\n자바,스프링부트,다양한 os, 다양한 db 에 대한 기초 지식이 필요합니다.\n\n\n조용필님은 기업메시징 엔진